In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as	plt
import seaborn as sns
import json

In [2]:
df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

C:\Users\Ephyrus\AppData\Local\Temp\ipykernel_26544\2090610779.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('train.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])


In [3]:
display(df)

,Token,Label
0,Estuvimos,O
1,hace,O
2,poco,O
3,mi,O
4,pareja,O
...,...,...
31171,junto,O
31172,con,O
31173,la,O
31174,carta,O


In [6]:
df_token = df.loc[:,["Token"]]
df_label = df.loc[:,["Label"]]

arr_token = (df_token.squeeze()).to_numpy()
arr_label = (df_label.squeeze()).to_numpy()

token_all = df_token.squeeze().unique()
label_all = df_label.squeeze().unique()

In [7]:
y_all = pd.DataFrame(df_label.value_counts()).reset_index()
y_all.columns = ['Label', 'Count']
y_all["Location"] = [np.where(arr_label == i)[0] for i in label_all]

print(len(y_all))
print(type(y_all))
display(y_all)

7
<class 'pandas.core.frame.DataFrame'>


,Label,Count,Location
0,O,29035,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,B-positive,1160,"[16, 19, 22, 25, 60, 63, 75, 86, 90, 113, 204,..."
2,B-negative,381,"[39, 98, 124, 131, 133, 137, 323, 382, 472, 62..."
3,I-positive,314,"[153, 591, 685, 1053, 1180, 1467, 1973, 1994, ..."
4,I-negative,171,"[154, 155, 156, 157, 158, 159, 160, 161, 162, ..."
5,B-neutral,72,"[313, 314, 315, 937, 1400, 1401, 1939, 2054, 2..."
6,I-neutral,43,"[802, 1171, 1626, 1627, 1724, 1725, 1726, 1727..."


In [8]:
x_all = pd.DataFrame(token_all, columns=['Token'])

# Need to check more, this takes >1m to do
new_x_all = pd.DataFrame({'Token': np.repeat(x_all['Token'], 7)}).reset_index()
new_x_all.columns = ['Index', 'Token']
new_x_all["Label"] = np.tile(label_all, int(len(new_x_all)/7))
count_x_all = []
location_x_all = []

for i in token_all:
    for j in label_all:
        cond_1 = arr_token == i
        cond_2 = arr_label == j
        loc = (np.where(cond_1 & cond_2)[0]).tolist()
        # df_ij = df.iloc[loc, :]
        count_x_all += [len(loc)]
        location_x_all += [loc]
new_x_all["Count"] = count_x_all
new_x_all["Location"] = location_x_all

print(len(new_x_all))
print(type(new_x_all))
display(new_x_all)

35280
<class 'pandas.core.frame.DataFrame'>


,Index,Token,Label,Count,Location
0,0,Estuvimos,O,6,"[0, 13026, 13936, 17095, 27870, 29879]"
1,0,Estuvimos,B-positive,0,[]
2,0,Estuvimos,B-negative,0,[]
3,0,Estuvimos,B-neutral,0,[]
4,0,Estuvimos,I-neutral,0,[]
...,...,...,...,...,...
35275,5039,inolvidable,B-negative,0,[]
35276,5039,inolvidable,B-neutral,0,[]
35277,5039,inolvidable,I-neutral,0,[]
35278,5039,inolvidable,I-positive,0,[]


$1. \:\:\: e(x \mid y)=\frac{\operatorname{Count}(y \rightarrow x)}{\operatorname{Count}(y)}$

In [9]:
def emm_par(x, y):
    # x is an instance of Token
    # y is an instance of Label
    cond_1 = new_x_all["Token"] == x
    cond_2 = new_x_all["Label"] == y
    num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0]
    return num/denom

In [10]:
a = emm_par("Estuvimos", "O")
print(a)

b = emm_par("estuvimos", "O")
print(b)

0.0002066471499913897
0.00030997072498708454


$2. \:\:\:e(x \mid y)=\left\{\begin{array}{cc}
\frac{\operatorname{count}(y \rightarrow x)}{\operatorname{Count}(y)+k} & \text { If the word token } x \text { appears in the training set } \\
\frac{k}{\operatorname{Count}(y)+k} & \text { If word token } x \text { is the special token \#UNK\# }
\end{array}\right.$

In [11]:
def emm_par_test(x, y):
    # x is an instance of Token
    # y is an instance of Label
    k = 1
    if x in new_x_all.loc[:, "Token"].unique():
        cond_1 = new_x_all["Token"] == x
        cond_2 = new_x_all["Label"] == y
        num = ((new_x_all[cond_1 & cond_2])["Count"]).values[0]
    else:
        num = k
    cond_3 = y_all["Label"] == y
    denom = ((y_all[cond_3])["Count"]).values[0] + k
    return num/denom

In [12]:
a = emm_par_test(",", "I-negative")
print(a)

b = emm_par_test("Kontollllllllllll", "I-neutral")
print(b)

0.01744186046511628
0.022727272727272728


$3. \:\:\:y^*=\underset{y}{\arg \max } \:e(x \mid y)$

In [13]:
df_y = pd.DataFrame()

emm_par_score_all = []
y_star_all = []
token_all = np.append(token_all, "#UNK#")
print(token_all)

['Estuvimos' 'hace' 'poco' ... 'redondeó' 'inolvidable' '#UNK#']


In [14]:
# Need to check more, this takes >2m to do
for i in token_all:
    emm_par_score = 0
    y_star = ""
    for j in label_all:
        emm_par_score_temp = emm_par_test(i,j)
        if emm_par_score_temp > emm_par_score:
            emm_par_score = emm_par_score_temp
            y_star = j
        else:
            pass
    y_star_all += [y_star]
    emm_par_score_all += [emm_par_score]

In [15]:
print(emm_par_score_all)
print(y_star_all)

[0.00020664003306240529, 0.0008954401432704229, 0.0018942003030720485, 0.0024796803967488635, 0.00044772007163521146, 0.0352665656426505, 0.005813953488372093, 0.00034440005510400884, 0.00013776002204160352, 0.005813953488372093, 0.013638242182118749, 0.005682600909216145, 0.05730816916930707, 0.0013431602149056344, 0.0380952380952381, 0.14556416881998277, 0.022727272727272728, 0.015706806282722512, 0.03789836347975883, 0.006029285099052541, 0.002617801047120419, 0.005028240804518529, 0.0003788400606144097, 0.005813953488372093, 0.055896128943380634, 0.0010332001653120264, 0.00017220002755200442, 0.00020664003306240529, 0.022727272727272728, 0.00013776002204160352, 0.005813953488372093, 0.012708362033337925, 0.0017908802865408459, 0.009298801487808239, 0.011627906976744186, 6.888001102080176e-05, 0.029101804656288744, 3.444000551040088e-05, 0.00017220002755200442, 0.00020664003306240529, 0.00017220002755200442, 6.888001102080176e-05, 0.023255813953488372, 6.888001102080176e-05, 0.00451

In [65]:
df_y["Token"] = token_all
df_y["y*"] = y_star_all
df_y["Max e(x|y)"] = emm_par_score_all

display(df_y)

for i in label_all:
    print(emm_par_test("sin", i))

,Token,y*,Max e(x|y)
0,Estuvimos,O,0.000207
1,hace,O,0.000895
2,poco,O,0.001894
3,mi,O,0.002480
4,pareja,O,0.000448
...,...,...,...
5036,informado,O,0.000034
5037,almuerzo,O,0.000034
5038,redondeó,O,0.000034
5039,inolvidable,O,0.000034


0.0028240804518528725
0.0
0.0
0.0
0.0
0.0
0.005813953488372093


In [21]:
df_y_dict = {row['Token']: (row['y*'], row['Max e(x|y)']) for row in df_y.to_dict(orient='records')}
print(df_y_dict)
print(df_y_dict[","])

{'Estuvimos': ('O', 0.00020664003306240529), 'hace': ('O', 0.0008954401432704229), 'poco': ('O', 0.0018942003030720485), 'mi': ('O', 0.0024796803967488635), 'pareja': ('O', 0.00044772007163521146), 'y': ('O', 0.0352665656426505), 'yo': ('I-negative', 0.005813953488372093), 'comiendo': ('O', 0.00034440005510400884), 'resultó': ('O', 0.00013776002204160352), 'todo': ('I-negative', 0.005813953488372093), 'muy': ('O', 0.013638242182118749), 'bien': ('O', 0.005682600909216145), ',': ('O', 0.05730816916930707), 'tanto': ('O', 0.0013431602149056344), 'la': ('I-positive', 0.0380952380952381), 'comida': ('B-positive', 0.14556416881998277), 'el': ('I-neutral', 0.022727272727272728), 'vino': ('B-negative', 0.015706806282722512), 'trato': ('B-positive', 0.03789836347975883), 'decoración': ('B-positive', 0.006029285099052541), '…': ('B-negative', 0.002617801047120419), 'nos': ('O', 0.005028240804518529), 'gustó': ('O', 0.0003788400606144097), 'mucho': ('I-negative', 0.005813953488372093), '.': ('O'

Testing

In [22]:
df_test = pd.read_csv('dev_out.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])

C:\Users\Ephyrus\AppData\Local\Temp\ipykernel_26544\2761620611.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_test = pd.read_csv('dev_out.txt', sep=' ', error_bad_lines=False, engine="python", quoting=3,  dtype=str, encoding='utf-8', names=["Token", "Label"])


In [23]:
display(df_test)

,Token,Label
0,Plato,B-positive
1,degustación,I-positive
2,:,O
3,un,O
4,poco,O
...,...,...
4307,sin,O
4308,dejarte,O
4309,la,O
4310,cartera,O


In [24]:
df_test_token = df_test.loc[:,["Token"]]
df_test_label = df_test.loc[:,["Label"]]

arr_test_token = (df_test_token.squeeze()).to_numpy()
arr_test_label = (df_test_label.squeeze()).to_numpy()

token_test_all = df_test_token.squeeze().unique()
label_test_all = df_test_label.squeeze().unique()

In [25]:
test_pred = []
new_token = []
for i in arr_test_token:
    try:
        pred = df_y_dict[i][0]
        test_pred += [pred]
    except:
        pred = df_y_dict["#UNK#"][0]
        test_pred += [pred]
        new_token += [i]

In [29]:
df_test["Predicted Label"] = test_pred
display(df_test)

arr_test_pred_label = ((df_test.loc[:,["Predicted Label"]]).squeeze()).to_numpy()

,Token,Label,Predicted Label
0,Plato,B-positive,B-negative
1,degustación,I-positive,I-positive
2,:,O,O
3,un,O,O
4,poco,O,O
...,...,...,...
4307,sin,O,I-negative
4308,dejarte,O,I-neutral
4309,la,O,I-positive
4310,cartera,O,I-neutral


In [34]:
entity_loc = (np.where(arr_test_label != "O")[0]).tolist()
pred_entity_loc = (np.where(arr_test_pred_label != "O")[0]).tolist()

In [59]:
print(entity_loc)
print(pred_entity_loc)

[0, 1, 13, 16, 56, 57, 58, 70, 81, 94, 97, 98, 99, 102, 103, 104, 107, 108, 109, 110, 114, 130, 131, 142, 159, 162, 170, 174, 193, 208, 209, 210, 224, 249, 258, 269, 276, 287, 320, 376, 413, 418, 419, 420, 421, 425, 431, 433, 436, 486, 514, 553, 561, 585, 587, 593, 596, 598, 640, 647, 650, 653, 689, 720, 740, 754, 755, 779, 782, 786, 826, 829, 830, 831, 838, 844, 877, 886, 889, 895, 899, 919, 923, 939, 940, 941, 942, 1019, 1029, 1034, 1036, 1140, 1146, 1157, 1158, 1159, 1160, 1161, 1167, 1177, 1185, 1266, 1293, 1298, 1306, 1310, 1454, 1463, 1483, 1508, 1516, 1517, 1520, 1523, 1526, 1574, 1590, 1601, 1616, 1637, 1638, 1639, 1642, 1645, 1664, 1665, 1701, 1704, 1776, 1777, 1801, 1810, 1815, 1829, 1842, 1843, 1844, 1860, 1866, 1868, 1885, 1890, 1897, 1901, 1934, 1940, 1949, 1956, 1979, 2039, 2044, 2049, 2074, 2169, 2183, 2217, 2218, 2219, 2225, 2236, 2241, 2246, 2257, 2272, 2280, 2294, 2295, 2296, 2299, 2318, 2418, 2421, 2427, 2433, 2434, 2435, 2462, 2463, 2464, 2506, 2508, 2533, 2537, 254

In [55]:
total_num_pred_e = len(pred_entity_loc)
total_num_gold_e = len(entity_loc)

df_test_entity = ((df_test.loc[entity_loc,:]).copy()).reset_index(drop=True)
display(df_test_entity)
df_test_entity_label = df_test_entity.loc[:,["Label"]]
df_test_entity_pred_label = df_test_entity.loc[:,["Predicted Label"]]

c_pred_entity_loc = (np.where((df_test_entity_label.squeeze()).to_numpy() == (df_test_entity_pred_label.squeeze()).to_numpy())[0]).tolist()
print(c_pred_entity_loc)

total_num_c_pred_e = len(c_pred_entity_loc)

,Token,Label,Predicted Label
0,Plato,B-positive,B-negative
1,degustación,I-positive,I-positive
2,restaurante,B-positive,B-negative
3,carne,B-positive,B-negative
4,sorbete,B-positive,B-positive
...,...,...,...
309,servicio,B-positive,B-positive
310,servicio,B-negative,B-positive
311,mesa,B-negative,B-negative
312,camareros,B-positive,B-negative


[1, 4, 5, 7, 9, 11, 15, 17, 20, 21, 22, 23, 26, 27, 28, 34, 36, 38, 39, 41, 42, 45, 46, 48, 53, 55, 56, 58, 59, 60, 61, 62, 63, 67, 69, 75, 76, 79, 80, 89, 98, 101, 104, 110, 112, 114, 118, 122, 123, 124, 128, 129, 131, 135, 136, 138, 139, 141, 142, 143, 144, 147, 148, 150, 151, 156, 159, 160, 162, 164, 166, 168, 170, 171, 178, 180, 181, 182, 183, 185, 186, 187, 200, 201, 202, 203, 206, 208, 210, 211, 217, 219, 221, 223, 226, 230, 233, 234, 235, 236, 239, 245, 246, 250, 251, 252, 255, 256, 257, 258, 263, 264, 267, 271, 272, 273, 274, 280, 283, 290, 294, 298, 300, 301, 303, 304, 309, 311, 313]


In [56]:
print(total_num_pred_e)
print(total_num_gold_e)
print(total_num_c_pred_e)

1690
314
129


$\text { Precision }=\frac{\text { Total number of correctly predicted entities }}{\text { Total number of predicted entities }}$

In [67]:
precision = total_num_c_pred_e/total_num_pred_e

$\text { Recall }=\frac{\text { Total number of correctly predicted entities }}{\text { Total number of gold entities }}$

In [66]:
recall = total_num_c_pred_e/total_num_gold_e

$F=\frac{2}{1 / \text { Precision }+1 / \text { Recall }}$

In [69]:
f_score = 2/((1/precision)+(1/recall))

In [70]:
print(precision)
print(recall)
print(f_score)

0.07633136094674556
0.410828025477707
0.1287425149700599
